In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import signal
from sklearn.utils import shuffle
import librosa as lb
from collections import Counter
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Augmentation - signal rolling
import glob
import os
import librosa
import soundfile as sf
path_original = "/content/synthetic"
path_aug1_roll ="/content/drive/MyDrive/animal_sound_augmented1/synthetic/roll/"
n=0
for filename in glob.glob(os.path.join(path_original, "*.*")):
  data = librosa.core.load(filename, sr=None)[0] #, sr=2000
  data_roll = np.roll(data, 8000)
  sf.write(path_aug1_roll+'synthetic_roll'+str(n)+'.wav', data_roll, 2000)
  n+=1

In [ ]:
pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Augmentation - pitch shifting
from pydub import AudioSegment
from numpy.random import uniform
from scipy import signal
path_original = "/content/synthetic"
path_aug1_shift ="/content/drive/MyDrive/animal_sound_augmented1/synthetic/shift/"
n=0
for filename in glob.glob(os.path.join(path_original, "*.*")):
  sound = AudioSegment.from_file(filename, format="aac")
  #For every sound type, I am using the method under below.
  #sound = AudioSegment.from_file(filename, format=filename[-3:])
  octaves = 0.5 # For decreasing, octave can be -0.5, -2 etc.
  new_sample_rate = int(sound.frame_rate * (2.0 ** octaves))
  hipitch_sound = sound._spawn(sound.raw_data, overrides={'frame_rate': new_sample_rate})
  hipitch_sound = hipitch_sound.set_frame_rate(2000)
  hipitch_sound.export(path_aug1_shift+'synthetic_shift'+str(n)+'.wav', format="wav")
  n+=1

In [ ]:
# load sound files as raw data
def load_data(directory_path):
    data = [] # list of all raw audio array

    for path in glob.glob(os.path.join(directory_path, "*")): # loop to load all files with the given name mentioned in exel file
      aud = path #full path to heart wav file
      dt , sr = lb.load(aud, sr=None) # dt : raw array and sr:sampling rate
      #print(sr)
      dt = lb.to_mono(dt) # convert to mono channel if it is double
      dt = dt.astype('float32')

      # resampling frequecy rate into fix 2000
      # number_of_samples = round(len(dt) * float(2000) / sr) # total samples in full signals
      # readings_resampled = signal.resample(dt, number_of_samples) # resample signal into calculated samples so getting 2000 sr.
      data.append(dt) # append each signal into list as raw audio data

    return np.array(data)

# geting data PCG sound samples
directory_path = '/content/all_synthetic'
raw_data= load_data(directory_path)
len(raw_data)

120

In [ ]:
# make Chunks all normal and abnormal and save all audios
import soundfile as sf
# path_normal = "/content/drive/MyDrive/Physionet_chunks_8sec/normal/"
# path_abnormal = "/content/drive/MyDrive/Physionet_chunks_8sec/abnormal/"
path_all_normal = "/content/drive/MyDrive/animal_sound_augmented1/synthetic/all_synthetic/"
def make_chunks(data):
  input_length = 2000*8 # 8 seconds
  n=0
  for i in range(0, len(data)): # loop through evey raw audio array
    if len(data[i]) > input_length: # if signal len is greater then 8 sec
      for t in range(0, len(data[i])-input_length, input_length):
        dx = data[i][t:t + input_length]
        sf.write(path_all_normal+'normal'+str(n)+'.wav', dx, 2000)
        n+=1

        temp2 = data[i][len(dx):]
        if len(temp2)<=input_length:#
          dx = temp2
          while (input_length>len(dx)):
            dif = abs(input_length - len(dx))     # get require extended len
            req_padd = dx[:dif]
            dx = np.append(dx, req_padd, axis=0)

    elif len(data[i] < input_length):
      dx = data[i]
      while (input_length>len(dx)):
        dif = abs(input_length - len(dx))     # get require extended len
        req_padd = dx[:dif]
        dx = np.append(dx, req_padd, axis=0)

    else:
      dx = data[i]

    sf.write(path_all_normal+'synthetic'+str(n)+'.wav', dx, 2000)
    n+=1


In [ ]:
make_chunks(raw_data)